## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c fake-news

 97% 45.0M/46.5M [00:03<00:00, 21.6MB/s]
100% 46.5M/46.5M [00:03<00:00, 13.4MB/s]


In [10]:
import zipfile

zip_ref = zipfile.ZipFile('/content/fake-news.zip', 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [11]:
import pandas as pd

In [12]:
df=pd.read_csv('/content/train.csv')

In [13]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
df.shape

(20800, 5)

In [15]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [17]:
###Drop Nan Values
df=df.dropna()


In [18]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [20]:
## Get the Dependent features
y=df['label']

In [21]:
X.shape

(18285, 4)

In [22]:
y.shape

(18285,)

In [23]:
import tensorflow as tf

In [24]:
tf.__version__

'2.18.0'

In [25]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [26]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [27]:
messages=X.copy()

In [30]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [31]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [32]:
messages.reset_index(inplace=True)

In [33]:
messages.head(10)

,level_0,index,id,title,author,text
0,0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [34]:
import nltk
import re
from nltk.corpus import stopwords

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [38]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [39]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr[1]

[1688, 1005, 3872, 1384, 2010, 161, 2423]

In [40]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [41]:
onehot_repr[1]

[1688, 1005, 3872, 1384, 2010, 161, 2423]

### Embedding Representation

In [42]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1571 4872 1583 ...    0    0    0]
 [1688 1005 3872 ...    0    0    0]
 [3698 3234 1552 ...    0    0    0]
 ...
 [3602 4572 3026 ...    0    0    0]
 [4273 2918 2071 ...    0    0    0]
 [3064  242 3843 ...    0    0    0]]


In [43]:
embedded_docs[1]

array([1688, 1005, 3872, 1384, 2010,  161, 2423,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, BatchNormalization

In [45]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Input(shape=(20,)))  # Input shape (sequence length)
model.add(Embedding(input_dim=5000, output_dim=40))  # Vocabulary size = 10000
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 40)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 20, 40)              │             160 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 257,061 (1004.14 KB)

 Trainable params: 256,781 (1003.05 KB)

 Non-trainable params: 280 (1.09 KB)

None


In [46]:
len(embedded_docs),y.shape

(18285, (18285,))

In [47]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [48]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [50]:
### Finally Training
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.7852 - loss: 0.4637 - val_accuracy: 0.9039 - val_loss: 0.4932
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9291 - loss: 0.1769 - val_accuracy: 0.9024 - val_loss: 0.2848
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9474 - loss: 0.1316 - val_accuracy: 0.9158 - val_loss: 0.2498
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9610 - loss: 0.1025 - val_accuracy: 0.9158 - val_loss: 0.2481
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9628 - loss: 0.0945 - val_accuracy: 0.9175 - val_loss: 0.2759
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9755 - loss: 0.0675 - val_accuracy: 0.9118 - val_loss: 0.2722
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9750 - loss: 0.0674 - val_accuracy: 0.9138 - val_loss: 0.2871
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9808 - loss: 0.0545 - val_accuracy

### Performance Metrics And Accuracy

In [51]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [52]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [53]:
from sklearn.metrics import confusion_matrix

In [54]:
confusion_matrix(y_test,y_pred)

array([[3147,  272],
       [ 267, 2349]])

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9106876553438277

In [56]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

